In [8]:
# Import libraries and list models
https://learn.microsoft.com/en-us/azure/cognitive-services/openai/tutorials/embeddings
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

OPENAI_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = OPENAI_KEY
openai.api_base = OPENAI_ENDPOINT
# openai.api_version = "2022-12-01"
openai.api_version = "2023-03-15-preview"

url = OPENAI_ENDPOINT + "/openai/deployments?api-version=" + openai.api_version
request_headers = {"api-key": OPENAI_KEY}
# 发送请求，并输出详细请求信息和返回信息
r = requests.get(url, headers=request_headers)
# 解析返回的json，提取 data中的 model和id，再以表格形式打印出来
# print(r.text)
data = r.json()
df = pd.DataFrame(data['data'])
# 只输出 id 和 model 2 列
df_id_model = df[['id','model']]
df_id_model

SyntaxError: invalid syntax (1329892447.py, line 2)

In [12]:
# load bill_sum_data.csv
# This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df=pd.read_csv(os.path.join(os.getcwd(),'bill_sum_data.csv')) 
df_bills = df[['text','summary', 'title']]
print(df_bills)

# data cleaning
#https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters
# pd.options.mode.chained_assignment = "None" 

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()  
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)
df_bills

                                                 text  \
0   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
1   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
2   SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...   
3   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
4   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
5   SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...   
6   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
7   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
8   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
9   SECTION 1. SHORT TITLE.\n\n    This Act may be...   
10  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
11  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
12  SECTION 1. FINDINGS.\n\n    The Congress finds...   
13  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
14  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
15  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
16  SECTION 1. SHORT TITLE.\n\n

C:\Users\xuefe\AppData\Local\Temp\ipykernel_24112\2702880437.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))


,text,summary,title
0,SECTION 1. SHORT TITLE. This Act may be cited ...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...
1,SECTION 1. SHORT TITLE. This Act may be cited ...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...
3,SECTION 1. SHORT TITLE. This Act may be cited ...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...
4,SECTION 1. SHORT TITLE. This Act may be cited ...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...
5,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...
6,SECTION 1. SHORT TITLE. This Act may be cited ...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...
7,SECTION 1. SHORT TITLE. This Act may be cited ...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...
8,SECTION 1. SHORT TITLE. This Act may be cited ...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013
9,SECTION 1. SHORT TITLE. This Act may be cited ...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993


NameError: name 'df_bills' is not defined